### Libraries

In [26]:
import numpy as np
import galois
from ecpy.curves import Curve, Point
from eth_hash.auto import keccak
from eth_keys import keys


### Setup

In [27]:
cv = Curve.get_curve('secp256k1')
gen = cv.generator

order = cv.order

In [28]:
pkey = 0x2839467304643546456

pub = pkey * gen

def pub_to_address(pub):
    concat_x_y = pub.x.to_bytes(32, byteorder='big') + pub.y.to_bytes(32, byteorder='big')
    return keccak(concat_x_y).hex()[24:]

# eth address
print("0x" + pub_to_address(pub))

0xc39595b7fe0d7d122410f38ba9d4671007d93b93


In [29]:
# Signing (Making a proof that pkey acknowledged that msg)
def sign(message, privateKey):

    if (type(message) == type("hi")):
        h = int.from_bytes(keccak(message.encode('utf-8')), 'big')
    else:
        h = int.from_bytes(keccak(message.to_bytes(32, 'big')), 'big')

    k = pow(0x128431824, 0x4244124, order)
    
    # R is a random point, cannot reverse-engineer k due to the ECDLP difficulty
    R = k*gen
    r = R.x

    # note that if k is known, then the private key can be reverse-engineered
    s = pow(k, -1, order) * (h + privateKey*r) % order

    return (r, s)

(r, s) = sign("$100", pkey) 
print(r, s)

84576065444967005099065822339142728499441734689764049427372598653897581749784 97174227634380922565079176658682006002095407341521945574888355189519552826370


In [30]:
# Verifying
def verify(message, r, s):
    if (type(message) == type("hi")):
        h = int.from_bytes(keccak(message.encode('utf-8')), 'big')
    else:
        h = int.from_bytes(keccak(message.to_bytes(32, 'big')), 'big')

    s_inv = pow(s, -1, order)

    R_used = ((h*s_inv)*gen + (r*s_inv)*pub)

    return (R_used.x == r)

print(verify("$100", r, s))
print(verify("$100", r, order - s))

True
True


In [31]:
# ecrecover (kinda since idk how the 'v' works yet)
def ecrecover_almost(message, r, s):

    if (type(message) == type("hi")):
        h = int.from_bytes(keccak(message.encode('utf-8')), 'big')
    else:
        h = int.from_bytes(keccak(message.to_bytes(32, 'big')), 'big')

    r_inv = pow(r, -1, order)
    R_y_0 = cv.y_recover(r, 0)
    R_y_1 = cv.y_recover(r, 1)

    R0 = Point(r, R_y_0, cv)
    R1 = Point(r, R_y_1, cv)

    Pub0 = r_inv*(s*R0 - h*gen)
    Pub1 = r_inv*(s*R1 - h*gen)

    return ["0x" + pub_to_address(Pub0), "0x" + pub_to_address(Pub1)]
    
print("the signer is either one of:\n", ecrecover_almost("$100", r, s))

the signer is either one of:
 ['0x0071544abbe8ecfe4014f44f0b0b852722517b57', '0xc39595b7fe0d7d122410f38ba9d4671007d93b93']
